In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session




# Tabular Data

In [2]:
train_data = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv', low_memory=False)
test_data = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv', low_memory=False)

In [3]:
useless_cols = set(train_data.columns) - set(test_data.columns)
useless_cols.remove('target')
useless_cols = list(useless_cols)
train_data = train_data.drop(columns=useless_cols)
train_size = train_data.shape[0]

In [4]:
X_train = train_data.drop(columns='target')
y_train = train_data['target']
X_train.fillna({'age_approx': X_train['age_approx'].mean(),'anatom_site_general':'NA','sex':'NA'}, inplace=True)

In [5]:
all_data = pd.concat([X_train, test_data])
all_data.reset_index(drop=True, inplace=True)
all_data = all_data.drop(columns=['copyright_license', 'attribution', 'image_type'])
# For second xgb model
all_data_2 = all_data.copy()

In [6]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
col_num = []
col_cat = []
for col in all_data:
  if col in ['isic_id', 'patient_id']: continue;
  if all_data[col].dtype == 'object' or all_data[col].dtype == 'character':
    col_cat.append(col)
    all_data[col] = all_data[col].astype('category')
  else:
    all_data[col] = all_data[col].astype('float64')
    col_num.append(col)

In [7]:
# Normalization for numeric values
numeric_transformer = StandardScaler()
all_data.loc[:,col_num] = numeric_transformer.fit_transform(all_data[col_num])

In [8]:
categorical_transformer = OneHotEncoder(sparse_output=False)
X_cat = categorical_transformer.fit_transform(all_data[col_cat])
new_cat_col = categorical_transformer.get_feature_names_out(col_cat)

In [9]:
all_data = all_data.drop(columns=col_cat)
all_data = pd.concat([all_data,pd.DataFrame(X_cat, columns=new_cat_col)],axis=1)
X_train = all_data.iloc[0:train_size,:]
X_test = all_data.iloc[train_size:,:]
X_test.reset_index(drop=True, inplace=True)

In [10]:
col_id = ['isic_id', 'patient_id']
X_train_id = X_train[col_id]
X_train_data = X_train.drop(columns=col_id)
X_test_id = X_test[col_id]
X_test_data = X_test.drop(columns=col_id)

In [11]:
import xgboost as xgb
import seaborn as sns

neg, pos = np.bincount(y_train)
scale_pos_weight = neg/pos
xgb_ml = xgb.XGBClassifier(
            scale_pos_weight=scale_pos_weight,  # Handle class imbalance
            learning_rate=0.01,
            n_estimators=200,
            max_depth=3,
            min_child_weight=4,
            gamma=1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            eval_metric=['logloss','aucpr']  # Use appropriate metric for binary classification
        )

In [12]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=41)
# X_train_pca = pca.fit_transform(X_train_data)
# X_test_pca = pca.transform(X_test_data)
# xgb_ml.fit(X_train_pca, y_train)
# predictions = xgb_ml.predict_proba(X_test_pca)[:, 1]

# def features_todrop(df, threshold=0.8):
#     corr_matrix = df.corr().abs()
#     upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
#     return  [column for column in upper.columns if any(upper[column] > threshold)]

# to_drop = features_todrop(X_train_data[col_num])
# X_train_data = X_train_data.drop(to_drop, axis=1)
# X_test_data = X_test_data.drop(to_drop, axis=1)
xgb_ml.fit(X_train_data, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=['logloss', 'aucpr'],
              feature_types=None, gamma=1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=4, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
all_data_2["lesion_size_ratio"] = all_data_2["tbp_lv_minorAxisMM"] / all_data_2["clin_size_long_diam_mm"]
all_data_2["lesion_shape_index"] = all_data_2["tbp_lv_areaMM2"] / (all_data_2["tbp_lv_perimeterMM"] ** 2)
all_data_2["hue_contrast"] = (all_data_2["tbp_lv_H"] - all_data_2["tbp_lv_Hext"]).abs()
all_data_2["luminance_contrast"] = (all_data_2["tbp_lv_L"] - all_data_2["tbp_lv_Lext"]).abs()
all_data_2["lesion_color_difference"] = np.sqrt(all_data_2["tbp_lv_deltaA"] ** 2 + all_data_2["tbp_lv_deltaB"] ** 2 + all_data_2["tbp_lv_deltaL"] ** 2)
all_data_2["border_complexity"] = all_data_2["tbp_lv_norm_border"] + all_data_2["tbp_lv_symm_2axis"]
all_data_2["3d_position_distance"] = np.sqrt(all_data_2["tbp_lv_x"] ** 2 + all_data_2["tbp_lv_y"] ** 2 + all_data_2["tbp_lv_z"] ** 2)
all_data_2["perimeter_to_area_ratio"] = all_data_2["tbp_lv_perimeterMM"] / all_data_2["tbp_lv_areaMM2"]
all_data_2["area_to_perimeter_ratio"] = all_data_2["tbp_lv_areaMM2"] / all_data_2["tbp_lv_perimeterMM"]
all_data_2["lesion_visibility_score"] = all_data_2["tbp_lv_deltaLBnorm"] + all_data_2["tbp_lv_norm_color"]
all_data_2["combined_anatomical_site"] = all_data_2["anatom_site_general"] + "_" + all_data_2["tbp_lv_location"]
all_data_2["symmetry_border_consistency"] = all_data_2["tbp_lv_symm_2axis"] * all_data_2["tbp_lv_norm_border"]
all_data_2["consistency_symmetry_border"] = all_data_2["tbp_lv_symm_2axis"] * all_data_2["tbp_lv_norm_border"] / (all_data_2["tbp_lv_symm_2axis"] + all_data_2["tbp_lv_norm_border"])
all_data_2["color_consistency"] = all_data_2["tbp_lv_stdL"] / all_data_2["tbp_lv_Lext"]
all_data_2["consistency_color"] = all_data_2["tbp_lv_stdL"] * all_data_2["tbp_lv_Lext"] / (all_data_2["tbp_lv_stdL"] + all_data_2["tbp_lv_Lext"])
all_data_2["size_age_interaction"] = all_data_2["clin_size_long_diam_mm"] * all_data_2["age_approx"]
all_data_2["hue_color_std_interaction"] = all_data_2["tbp_lv_H"] * all_data_2["tbp_lv_color_std_mean"]
all_data_2["lesion_severity_index"] = (all_data_2["tbp_lv_norm_border"] + all_data_2["tbp_lv_norm_color"] + all_data_2["tbp_lv_eccentricity"]) / 3
all_data_2["shape_complexity_index"] = all_data_2["border_complexity"] + all_data_2["lesion_shape_index"]

# all_data_2["color_contrast_index"] = all_data_2["tbp_lv_deltaA"] + all_data_2["tbp_lv_deltaB"] + all_data_2["tbp_lv_deltaL"] + all_data_2["tbp_lv_deltaLBnorm"]
# all_data_2["log_lesion_area"] = np.log(all_data_2["tbp_lv_areaMM2"] + 1)
# all_data_2["normalized_lesion_size"] = all_data_2["clin_size_long_diam_mm"] / all_data_2["age_approx"]
# all_data_2["mean_hue_difference"] = (all_data_2["tbp_lv_H"] + all_data_2["tbp_lv_Hext"]) / 2
# all_data_2["std_dev_contrast"] = np.sqrt((all_data_2["tbp_lv_deltaA"] ** 2 + all_data_2["tbp_lv_deltaB"] ** 2 + all_data_2["tbp_lv_deltaL"] ** 2) / 3)
# all_data_2["color_shape_composite_index"] = (all_data_2["tbp_lv_color_std_mean"] + all_data_2["tbp_lv_area_perim_ratio"] + all_data_2["tbp_lv_symm_2axis"]) / 3
# all_data_2["3d_lesion_orientation"] = np.arctan2(all_data_2["tbp_lv_y"], all_data_2["tbp_lv_x"])
# all_data_2["overall_color_difference"] = (all_data_2["tbp_lv_deltaA"] + all_data_2["tbp_lv_deltaB"] + all_data_2["tbp_lv_deltaL"]) / 3
# all_data_2["symmetry_perimeter_interaction"] = all_data_2["tbp_lv_symm_2axis"] * all_data_2["tbp_lv_perimeterMM"]
# all_data_2["comprehensive_lesion_index"] = (all_data_2["tbp_lv_area_perim_ratio"] + all_data_2["tbp_lv_eccentricity"] + all_data_2["tbp_lv_norm_color"] + all_data_2["tbp_lv_symm_2axis"]) / 4
# all_data_2["color_variance_ratio"] = all_data_2["tbp_lv_color_std_mean"] / all_data_2["tbp_lv_stdLExt"]
# all_data_2["border_color_interaction"] = all_data_2["tbp_lv_norm_border"] * all_data_2["tbp_lv_norm_color"]
# all_data_2["border_color_interaction_2"] = all_data_2["tbp_lv_norm_border"] * all_data_2["tbp_lv_norm_color"] / (all_data_2["tbp_lv_norm_border"] + all_data_2["tbp_lv_norm_color"])
# all_data_2["size_color_contrast_ratio"] = all_data_2["clin_size_long_diam_mm"] / all_data_2["tbp_lv_deltaLBnorm"]
# all_data_2["age_normalized_nevi_confidence"] = all_data_2["tbp_lv_nevi_confidence"] / all_data_2["age_approx"]
# all_data_2["age_normalized_nevi_confidence_2"] = np.sqrt(all_data_2["clin_size_long_diam_mm"]**2 + all_data_2["age_approx"]**2)
# all_data_2["color_asymmetry_index"] = all_data_2["tbp_lv_radial_color_std_max"] * all_data_2["tbp_lv_symm_2axis"]
# all_data_2["volume_approximation_3d"] = all_data_2["tbp_lv_areaMM2"] * np.sqrt((all_data_2["tbp_lv_x"]**2 + all_data_2["tbp_lv_y"]**2 + all_data_2["tbp_lv_z"]**2))
# all_data_2["color_range "] = (all_data_2["tbp_lv_L"] - all_data_2["tbp_lv_Lext"]).abs() + (all_data_2["tbp_lv_A"] - all_data_2["tbp_lv_Aext"]).abs() + (all_data_2["tbp_lv_B"] - all_data_2["tbp_lv_Bext"]).abs()
# all_data_2["shape_color_consistency"] = all_data_2["tbp_lv_eccentricity"] * all_data_2["tbp_lv_color_std_mean"]
# all_data_2["border_length_ratio"] = all_data_2["tbp_lv_perimeterMM"] / (2 * np.pi * np.sqrt(all_data_2["tbp_lv_areaMM2"] / np.pi))
# all_data_2["age_size_symmetry_index"] = all_data_2["age_approx"] * all_data_2["clin_size_long_diam_mm"] * all_data_2["tbp_lv_symm_2axis"]
# all_data_2["index_age_size_symmetry"] = all_data_2["age_approx"] * all_data_2["tbp_lv_areaMM2"] * all_data_2["tbp_lv_symm_2axis"]

In [ ]:
all_data_2 = all_data_2.drop(columns=col_id)
X_train_2 = all_data_2.iloc[0:train_size,:]
X_test_2 = all_data_2.iloc[train_size:,:]
X_test_2.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, col_num),
        ('cat', cat_transformer, col_cat)
    ]
)

best_xgb_params = {
    'colsample_bytree': 0.8659,
    'gamma': 0.1787,
    'learning_rate': 0.0214,
    'max_depth': int(4.9),  # Convert to int as required by XGBoost
    'min_child_weight': 17.38,
    'n_estimators': int(130.67),  # Convert to int as required by XGBoost
    'reg_alpha': 0.8864,
    'reg_lambda': 0.3854,
    'subsample': 0.9212,
    'random_state': 42
}

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
# Preprocess the training data
X_train_preprocessed = preprocessor.fit_transform(X_train_2)

# Apply SMOTE
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_train_preprocessed, y_train)

# Train the XGBoost model with best parameters
final_xgb_model = XGBClassifier(**best_xgb_params)
final_xgb_model.fit(X_resampled, y_resampled)

# Preprocess the test data
X_test_preprocessed = preprocessor.transform(X_test_2)

# Predict on the test data
xgb_prob_test_2 = final_xgb_model.predict_proba(X_test_preprocessed)

# Image - ViT

In [13]:
root_dir = '/kaggle/input/isic-2024-challenge'

In [14]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms

def remove_hair(image):
    """
    Remove hair artifacts from an image using the DullRazor approach.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, thresh = cv2.threshold(blackhat, 12, 255, cv2.THRESH_BINARY)
    inpainted = cv2.inpaint(image, thresh, inpaintRadius=1, flags=cv2.INPAINT_TELEA)

    return inpainted

In [15]:
target_data = train_data[['isic_id', 'target']]

In [16]:
print(target_data.shape)

(401059, 2)


In [17]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import os

class CassavaDataset(torch.utils.data.Dataset):
    """
    Helper Class to create the PyTorch dataset
    """

    def __init__(self, df, data_path=f'{root_dir}/train-image/image/', secondary_data_path="/content/isic-2024-challenge/isic-2020-dataset/train/malignant", mode="train", transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.secondary_data_path = secondary_data_path
        self.transforms = transforms
        self.mode = mode

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name, label = self.df_data[index]
        if img_name.startswith("ISIC2020_"):
            stripped_name = img_name.replace("ISIC2020_", "")
            img_path = os.path.join(self.secondary_data_path, f"{stripped_name}")
        else:
            img_path = os.path.join(self.data_path, f"{img_name}.jpg")
        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            img = self.transforms(img)

        return img, label


In [18]:
class HairRemovalTransform:
    def __call__(self, img):
        img_np = np.array(img)
        img_np = remove_hair(img_np)
        img = Image.fromarray(img_np)
        return img

IMG_SIZE = 224

transforms_train = transforms.Compose(
    [
        HairRemovalTransform(),
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomResizedCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

In [19]:
import timm


class ViTBase16(nn.Module):
    def __init__(self, n_classes, pretrained=False, model_path=None):
        super(ViTBase16, self).__init__()
        # self.model = timm.create_model("vit_small_patch16_224", pretrained=False)
        self.model = timm.create_model("vit_large_patch16_224", pretrained=False)
        if pretrained and model_path:
            self.model.load_state_dict(torch.load(model_path))
        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return torch.sigmoid(x)

    def train_one_epoch(self, train_loader, criterion, optimizer, device):
        self.model.train()
        epoch_loss = 0.0
        epoch_accuracy = 0.0

        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = self(data)
            loss = criterion(output, target)
            loss.backward()
            accuracy = (output.argmax(dim=1) == target).float().mean()

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()

            optimizer.step()
            if device.type == "xla" and i % 20 == 0:
                xm.master_print(f"\tBATCH {i+1}/{len(train_loader)} - LOSS: {loss.item():.4f}")

        avg_loss = epoch_loss / len(train_loader)
        avg_accuracy = epoch_accuracy / len(train_loader)
        return avg_loss, avg_accuracy

    def validate_one_epoch(self, valid_loader, criterion, device):
        self.model.eval()
        valid_loss = 0.0
        valid_accuracy = 0.0

        with torch.no_grad():
            for data, target in valid_loader:
                data, target = data.to(device), target.to(device)

                output = self(data)
                loss = criterion(output, target)

                accuracy = (output.argmax(dim=1) == target).float().mean()

                valid_loss += loss.item()
                valid_accuracy += accuracy.item()

        avg_loss = valid_loss / len(valid_loader)
        avg_accuracy = valid_accuracy / len(valid_loader)
        return avg_loss, avg_accuracy

In [20]:
from io import BytesIO
class HDF5CassavaDataset(torch.utils.data.Dataset):

    def __init__(self, df, hdf5_loader, mode="test", transforms=None):
        super().__init__()
        self.df = df
        self.hdf5_loader = hdf5_loader
        self.mode = mode
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        isic_id = row['isic_id']
        # img_bytes = self.hdf5_loader.__getitem__(isic_id)
        img_bytes = self.hdf5_loader.get_img(isic_id)
        img = Image.open(BytesIO(img_bytes)).convert("RGB")

        if self.transforms:
            img = self.transforms(img)

        if self.mode in ["train", "valid"]:
            label = row['target']
            return img, label
        else: 
            return img, isic_id 

In [21]:
# dataloader
class UltDataLoader():
    def __init__(self, data_path, batch_size=32):
        self.data_path = data_path
        self.batch_size = batch_size
        self.index = 0

    def load_meta(self):
        for _ in ['train', 'test']:
            target_path = f"{self.data_path}/{_}-metadata.csv"
            setattr(self, f"{_}_meta", pd.read_csv(target_path,low_memory=False))
            
    def load_hdf5(self):
        for _ in ['train', 'test']:
            target_path = f"{self.data_path}/{_}-image.hdf5"
            setattr(self, f"{_}_hdf5", h5py.File(target_path, 'r'))


    def __getitem__(self, isic_id):
        if self.train_hdf5.get(isic_id):
            # seeking from train at first
            return self.train_hdf5[isic_id][()]
        elif self.test_hdf5.get(isic_id):
            # seeking from test if isic_id doesn't exit in train
            return self.test_hdf5[isic_id][()]
        else:
            raise ValueError(f"ISIC_ID {isic_id} is not found.")

    def get_img(self, isic_id, test=True):
        if test:
            # seeking from train at first
            return self.test_hdf5[isic_id][()]
        else:
            # seeking from test if isic_id doesn't exit in train
            return self.train_hdf5[isic_id][()]
    
    def hdf5_dataset2img(self, isic_id):
        """
        - We can access image data with `dataset[()]`.
        - dataset -> hdf5 dataset object
        - hdf5 dataset object -> We can extract with isic_id as key from h5py.File(path/to/file.hdf5, 'r')
          e.g) h5py_file[idic_id]
        
        Example:
            >>> import h5py
            >>> from io import BytesIO
            >>> from PIL import Image
            >>> import matplotlib.pyplot as plt
            >>> with h5py.File('/kaggle/input/isic-2024-challenge/train-image.hdf5') as f:
            >>>    plt.imshow(Image.open(BytesIO(f['ISIC_0015670'][()])))
        """
        img_bytes = self.__getitem__(isic_id)
        return BytesIO(img_bytes)

        

transforms_valid = transforms.Compose(
    [
        HairRemovalTransform(),
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

transforms_test = transforms.Compose([
    HairRemovalTransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [22]:
from torch.utils.data import DataLoader
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import h5py


hdf5_loader = UltDataLoader(root_dir)
hdf5_loader.load_meta()
hdf5_loader.load_hdf5()
test_meta = hdf5_loader.test_meta

train_dataset = CassavaDataset(target_data, transforms=transforms_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8)

# valid_dataset = CassavaDataset(val_meta, transforms=transforms_valid)
# valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=8)

test_dataset = HDF5CassavaDataset(test_meta, hdf5_loader, mode="test", transforms=transforms_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=8)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [24]:
model = ViTBase16(n_classes=2, pretrained=True)
md_path0 = '/kaggle/input/best-model-4/best_model_forth.pth'
md_path1 = '/kaggle/input/best-model-5/best_model_fifth.pth'
# checkpoint = torch.load(md_path0)
checkpoint = torch.load(md_path1)
model.load_state_dict(checkpoint)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# train_loss, train_acc = model.train_one_epoch(train_loader, criterion, optimizer, device)
# valid_loss, valid_acc = model.validate_one_epoch(valid_loader, criterion, device)

/tmp/ipykernel_24/2525441830.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(md_path0, map_location=torch.device('cpu'))


ViTBase16(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (act): GELU(approximate='no

In [25]:
from torch.nn.functional import softmax

In [ ]:
import h5py
import matplotlib.pyplot as plt
import os
import random
import numpy as np
from PIL import Image
from torchvision import transforms
import pandas as pd
import kagglehub
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import time
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, classification_report
import io
from io import BytesIO
import cv2
from torchvision import transforms
import shutil

root_dir = '/kaggle/input/isic-2024-challenge'
model_v2 = models.mobilenet_v2(weights=None)
model_v2.classifier[1] = torch.nn.Linear(in_features=1280, out_features=1) 
for param in model_v2.parameters():
    param.requires_grad = False

checkpoint_path = "/kaggle/input/mobilenet-v2/model_checkpoint_v2.pth"
checkpoint = torch.load(checkpoint_path)

model_v2.load_state_dict(checkpoint['model_state_dict'])

model_v2.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_v2.to(device)

probabilities_mobilenet = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model_v2(inputs).squeeze()
        probs = torch.sigmoid(outputs)
        probabilities_mobilenet.extend(probs.cpu().numpy())

In [26]:
def extract_features(model, dataloader, device, test=False):
    probabilities = []
    with torch.no_grad():  # No need to calculate gradients during inference
        for data, target in dataloader:
            # print(data)
            # print(target)
            data = data.to(device)
            if not test:
                target = target.to(device)  # Send target to device if necessary
            outputs = model(data)
        
            # Convert logits to probabilities using softmax
            probs = softmax(outputs, dim=1)  # Shape: [batch_size, n_classes]
            # print(probs)
            probabilities.extend(probs.cpu().numpy())
            
    return probabilities

# Ensemble 

In [4]:
import pandas.api.types
from sklearn.metrics import roc_curve, auc, roc_auc_score

class ParticipantVisibleError(Exception):
    pass


def score(solution, submission, min_tpr: float=0.80) -> float:

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution)-1)

    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

    return(partial_auc)

In [ ]:
# Ensure the model is on the correct device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# xgb_prob_train = xgb_ml.predict_proba(X_train_data)
# vit_prob_train = extract_features(model, train_loader, device)

xgb_prob_test = xgb_ml.predict_proba(X_test_data)
vit_prob_test = extract_features(model, test_loader, device, test=True)
# Combine features
# train_combined_prob = np.hstack([vit_prob_train, xgb_prob_train])
# test_combined_prob = np.hstack([vit_prob_test, xgb_prob_test])

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
from scipy.optimize import minimize

# xgb_prob_train_1 = xgb_prob_train[:,1]
# vit_prob_train_1 = np.array([prob[1] for prob in vit_prob_train])

pred1 = xgb_prob_test[:,1]
pred2 = xgb_prob_test_2[:,1]
pred3 = np.array([prob[1] for prob in vit_prob_test])
pred4 = np.array(probabilities_mobilenet)
preds = [pred1,pred2,pred3,pred4]

In [ ]:
# Predict
# final_preds = np.sum(result.x.reshape(-1, 1) * prob_test_1, axis=0)

final_preds = (0.3*pred1+0.3*pred2+0.3*pred3+0.1*pred4)/4
# final_preds = np.power(pred1*pred2*pred3, 1/3)
# Logit Average
# Convert probabilities to log-odds
# log_odds = [np.log(p / (1 - p)) for p in preds]

# # Average the log-odds
# average_log_odds = np.mean(log_odds, axis=0)

# # Convert back to probabilities
# final_preds = 1 / (1 + np.exp(-average_log_odds))

submission = pd.DataFrame(
    {'isic_id': X_test['isic_id'], 'target': final_preds},
    columns = ['isic_id', 'target'])
submission.to_csv('/kaggle/working/submission.csv', index = False)